In [ ]:
### limites actuelles et TODO du script:
# TODO: rajout github et d'autres liens
# TODO: tester le rajout de nouveaux profiles (faut-il lister quelques parts les fichiers? ou automatiquement récupérés?)
# limite: la clé pour regrouper des informations est une partie du nom/le nom de l'EIG
# limite: écrasement du fichier précédent (nécessité de rentrer les défis précédents à main)
# limite: les fichiers sources ne sont pas rajouté. S'il le faut, il faut filtrer l'information sensible dessus avant de le faire

In [ ]:
import pandas as pd
import numpy as np
import os


# information entrée par les participants
df = pd.read_excel(
     os.path.join('./bienvenue_dans_le_programme_eig_porteurs.xlsx'),
     engine='openpyxl',
    skiprows=[0,1]
)

df_sup = pd.read_csv('./promotion_EG6.csv', sep='\t')


# variables: à ajuster
promotion_numero = "6"
role1 = "agent-public" 

In [ ]:
### Reformattage
# df
df['Nom'] = df['Nom'].str.title()
df['Prénom'] = df['Prénom'].replace(['bboris', 'Raphaële '], ['Boris', 'Raphaële'])
df['title'] = df['Prénom'] + ' ' + df['Nom']
# nom du fichier .md
df['nom_du_fichier'] = df['Prénom'].str.lower().replace(' ', '-') + '-' + df['Nom'].str.lower().str.replace(' ', '-') + '.md'
df = df.rename(columns= {'URL de votre profil LinkedIn': 'linkedin'})
df['linkedin'] = df['linkedin'].replace('https://fr.linkedin.com › françois-lecordix-b55598147', 'https://www.linkedin.com/in/fran%C3%A7ois-lecordix-b55598147/')
# photo:
df = df.rename(columns= {'Nom.1': 'Photo'})

#df_sup: dernière partie du nom de famille comme clé
df_sup['Nom'] = df_sup['Nom'].str.replace('-', ' ').str.split(' ').str[-1].str.title().replace(['Heno', 'Belorgey'],  ['Héno','Bélorgey'] )
# standardiser
df_sup['Défi'] = df_sup['Défi'].replace(['artificIA', 'Écosphère(s)', "Estim'action", 'État bas carbone'], ['ArtificIA', 'Ecosphère(s)', "Estim'Action", 'Etat bas carbone'])


# regrouper les informations (clé: Nom)
df['Nom'] = df['Nom'].str.replace('ï', 'i').str.replace('-', ' ').str.split(' ').str[-1] # comme pour df_sup
df_merged = df.merge(df_sup, left_on='Nom', right_on='Nom')

In [ ]:
# Ecriture des fichiers
def is_nan(x): #helper
    return (x != x)


for i in range(df_merged.shape[0]):
    line = df_merged.loc[i] # donnée d'un(e) EIG
    with open('./PycharmProjects/site-eig-v2/site/content/personnes/'+line['nom_du_fichier'], "w") as curseur:
        
        # entête:
        # lignes formatées: nom, métier, promotion, nom du défi, expertise, image, lien linkedin
        curseur.write('---' + '\n')
        
        curseur.write('title: {}\n'.format(line['title']))
        
        curseur.write('job: {}\n'.format(line['Role']))
        
        curseur.write('promos:\n')
        curseur.write('  - Promotion {}\n'.format(promotion_numero))
        
        curseur.write('role: {}\n'.format(role1))
        
        curseur.write('defis:\n')
        curseur.write('  - {}\n'.format(line['Défi']))
        
        #curseur.write('expertise:\n')
        #curseur.write('  - {}\n'.format(line['Expertises']))
        
        curseur.write('image: img/promotion-6-photos/{}\n'.format(line['Photo'].split('/')[-1]))
        
        if not is_nan(line['linkedin']):
            curseur.write('linkedin: {}\n'.format(line['linkedin']))
        
        curseur.write('---\n\n')
        # description:
        curseur.write('{}\n'.format(line['Description']))